In [1]:
import pandas as pd
import os

%load_ext autoreload
%autoreload 2 

In [2]:
import sys
sys.path.insert(0, '..')
import os
os.chdir('..')

In [ ]:
print(sys.path)

['..', '..', '/Users/yenchunchen/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python312.zip', '/Users/yenchunchen/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12', '/Users/yenchunchen/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/lib-dynload', '', '/Users/yenchunchen/Desktop/Project/health-agent/.venv/lib/python3.12/site-packages', '/Users/yenchunchen/Desktop/Project/health-agent/service/youtube_extraction/src', '/var/folders/t7/tf0dc46s4hdc5zgvcgk2vhd80000gn/T/tmppqusc957']


In [3]:
from dotenv import load_dotenv
load_dotenv()
from pydantic_ai import Agent
from pydantic import BaseModel, Field

In [4]:
!pwd

/Users/yenchunchen/Desktop/Project/health-agent


In [7]:
# from service.product_extraction.src.product_extraction.sephora_search_tool import prepare_sephora_search_tool, SephoraSearchTools
from service.youtube_extraction.src.youtube_extraction.youtube_search import YoutuberTranscriptSearcher

/Users/yenchunchen/Desktop/Project/health-agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from pydantic import BaseModel, Field
from typing import Optional, Dict, List


def has_cached_transcript(youtuber, video_id, window_size: int = 15, step_size: int = 3):
    """
    Check whether cached transcript chunks exist for a given YouTube video.

    The function looks for a JSON file named
        {youtuber}_{video_id}.json
    inside the directory:
        {CACHE_DATA_DIR}/{youtuber}_{window_size}_{step_size}/

    Args:
        youtuber (str): The YouTuber's name used for folder and file naming.
        video_id (str): The YouTube video ID.
        window_size (int): Sliding window size used during transcript chunking.
        step_size (int): Step size used during transcript chunking.

    Returns:
        bool: True if the cached transcript file exists, False otherwise.
    """

    cache_dir = CACHE_DATA_DIR / Path(f"{youtuber}_{window_size}_{step_size}_Chinese")
    file_path = cache_dir / f"{youtuber}_{video_id}.json"

    return file_path.exists()

import json

def list_cached_videos(youtuber: str, window_size: int = 15, step_size: int = 3) -> List[dict]:
    """
    List all cached video IDs for a given YouTuber.

    Args:
        youtuber (str): Name of the YouTuber.
        window_size (int): Sliding window size used during processing.
        step_size (int): Step size used during processing.

    Returns:
        List[dict]: List of title, summary, video IDs available in the cache for this YouTuber.
    """
    
    cache_dir = CACHE_DATA_DIR / f"{youtuber}_{window_size}_{step_size}_Chinese"

    if not cache_dir.exists():
        return []

    video_url = []
    for file in cache_dir.glob(f"{youtuber}_*.json"):
        with open(file, "r", encoding="utf-8") as f:
            chunks = json.load(f)
        summary = chunks[0]['summary']
        title = chunks[0]['title']
        # Extract video_id from filename
        name = file.stem  # e.g., "heyitsmindy_9QSmr_2EGXM"
        parts = name.split(f"{youtuber}_")
        if len(parts) == 2:
            video = {
                'title': title,
                'summary': summary,
                'url': f"https://www.youtube.com/watch?v={parts[1]}"
            }
            video_url.append(video)
    return video_url

In [14]:
from minsearch import Index
from pathlib import Path

CACHE_DATA_DIR = Path(".cache")

In [15]:
has_cached_transcript(youtuber='heyitsmindy', video_id='9QSmr_2EGXM')

True

In [16]:
list_cached_videos(youtuber='heyitsmindy')

[{'title': 'Sephora Summary: What to Buy and What Not to!',
  'summary': "In this video, Mindy shares various products she has purchased from Sephora over the past few years, including lip balms, primers, foundations, concealers, setting powders, blushes, and lip colors. She emphasizes the use and effects of several products, especially recommending the Topicals lip balm, Tower 28's SOS primer spray, and Natasha Denona's Hy-Glam foundation, believing these products are suitable for Asian skin tones and worth purchasing during sales. The video also mentions products she does not recommend, such as Charlotte Tilbury's Airbrush foundation and certain brands of blushes, highlighting their shortcomings in coverage and finish. At the end of the video, Mindy hopes viewers find it helpful and looks forward to their feedback.",
  'url': 'https://www.youtube.com/watch?v=2SvN45DKWFg'},
 {'title': 'Korean Makeup New Product Review',
  'summary': "In this video, Mindy shares her experiences with a 

In [70]:
from service.youtube_extraction.src.youtube_extraction.youtube_search import YoutuberTranscriptSearcher

In [71]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:

    def __init__(self, agent):
        self.agent_name = agent.name

    async def print_function_calls(self, ctx, event):
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub in event:
                await self.print_function_calls(ctx, sub)
            return

        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

    async def __call__(self, ctx, event):
        return await self.print_function_calls(ctx, event)

In [194]:

class Videos(BaseModel):
    title: str = Field(... , description='Title of the cached video.')
    summary: str = Field(... , description='Summary of the cached video.')
    url: str = Field(... , description='Youtuber URL for the cached video')



class ClarifyDecision(BaseModel):
    """
    Routing + query understanding for the Beauty YouTube system.
    """

    # Understanding the request
    youtuber: str = Field(..., description="The Youtuber mentioned or inferred from the user query.")
    user_intent: str = Field(..., description="The goal of the user: e.g. 'find_product', 'recommendation', 'compare', 'video_question'.")
    topic: Optional[str] = Field(None, description="Product or topic extracted from the user query.")
    video_id: Optional[str] = Field(None, description="YouTube video ID if specified, else None.")

    # Cache and data routing
    in_cache: bool = Field(..., description="Whether the transcript for this video exists in the cache.")
    # use_cache_only: bool = Field(..., description="If True, system should use cached data only.")
    # requires_online_search: bool = Field(..., description="If True, system must fetch the transcript online and chunk it.")
    # add_to_cache: bool = Field(..., description="If True, system must chunk, summarize, and save the transcript.")

    # When video_id is missing
    # ask_for_video: bool = Field(..., description="If True, ask the user to pick a video.")
    available_videos: Optional[List[Videos]] = Field(
        None,
        description="List of cached videos (title, summary, url) to show user when no video_id provided."
    )

    def print_agent_output(self):
        """
        Print the full Clarify Agent output in a readable format.

        Args:
            decision (ClarifyDecision): The ClarifyDecision object returned by the agent.
        """
        print("=== Clarify Agent Output ===")
        print(f"Intent       : {self.user_intent}")
        print(f"YouTuber     : {self.youtuber}")
        print(f"Video ID     : {self.video_id}")
        print(f"Topic        : {self.topic}")
        # print(f"Add Data     : {decision.add_to_cache}")
        print(f"In Cache  : {self.in_cache}\n")

        # if decision.ask_for_video:
        
        if True:
            
            print("Hey I can start answer your question!\nHere are Available Videos (metadata):\n")
            for idx, video in enumerate(self.available_videos, start=1):
                print(f"{idx}. Title  : {video.title}")
                print(f"   Summary: {video.summary}")
                print(f"   URL    : {video.url}\n")
        elif self.available_videos:
            print("Available Video IDs:")
            for vid in decision.available_videos:
                print(f"- {vid}")
        else:
            print("No available videos to display.")
        print("============================\n")
    
# print_agent_output(result.output)


clarify_instruction = """

You are the Clarify Agent for a Beauty & YouTube Review Search System.

Your ONLY responsibilities are:
1. Understanding the user's message.
2. Determining whether to use cached transcript data or fetch/index new data.
3. Routing the flow correctly using the ClarifyDecision schema.

You MUST NOT:
- Answer beauty questions.
- Recommend products.
- Summarize any video.
- Pull transcript content.
You ONLY perform routing and decision-making.

====================================================
PRIMARY RULE — Cache First
====================================================
Always check if the transcript for the video is in the cache **before doing anything else**.
- If the transcript exists in cache → do NOT call chunk_transcript.
- If the transcript does NOT exist in cache → always call chunk_transcript to generate and save the chunks.
- After chunking, **call `list_cached_videos(youtuber)`** to return all available videos in the cache for this YouTuber.

====================================================
1. QUERY UNDERSTANDING
====================================================
From the user's message, extract:

- youtuber:
    The content creator mentioned or implied.
    If none is present, return an empty string "".

- user_intent:
    Pick one of the following:
        - "find_product"          → Asking about a product or item.
        - "recommendation"        → Asking what to buy, or for suggestions.
        - "compare_products"      → Comparing two or more products.
        - "video_question"        → Asking for info about a specific video.
        - "index_video"           → Asking to ingest/index a specific video.
    Choose the best match based on user language.

- topic:
    A product or category mentioned (e.g., "foundation", "lip gloss").
    If none is mentioned, return null.

- video_id:
    Extract the YouTube video ID from:
        - Full URLs, e.g., "https://www.youtube.com/watch?v=2SvN45DKWFg"
        - Share links, e.g., "https://youtu.be/2SvN45DKWFg"
        - Raw video IDs, e.g., "2SvN45DKWFg"
    If no video ID is found → return null.

====================================================
2. ROUTING LOGIC
====================================================

-----------------------------------------
CASE A — The user does NOT provide a video_id
-----------------------------------------
If no video_id can be extracted:
    - Set:
        video_id = null
        in_cache = false
        user_intent = "no_video_id"
        available_videos = the provided list of cached videos
    - Do NOT attempt to guess, search, or index.

-----------------------------------------
CASE B — The user DOES provide a video_id
-----------------------------------------
If video_id is provided:
    - Call `has_cached_transcript(video_id)` exactly ONCE.
    - Set:
        in_cache = True  → if transcript is already cached  
        in_cache = False → if transcript does not exist

    - If in_cache = True:
        - Do NOT index again.
        - Prepare decision output immediately.

    - If in_cache = False:
        - Mark user_intent = "index_video" if user requested indexing explicitly
        - Or keep original intent but indicate in_cache = False
        - **ALWAYS Call the method `chunk_transcript(video_id)`** to:
            - Load the raw transcript
            - Split transcript into segments
            - Generate LLM-based summary
            - Build sliding-window chunks
            - Save chunks to cache
    - Only call `chunk_transcript` **once per request*

====================================================
3. OUTPUT FORMAT
====================================================
You MUST return a JSON object exactly matching the ClarifyDecision schema:

class ClarifyDecision(BaseModel):
    youtuber: str
    user_intent: str
    topic: Optional[str]
    video_id: Optional[str]
    in_cache: bool
    available_videos: Optional[List[Videos]]

- Do NOT answer the user.
- Do NOT comment on the query.
- ONLY produce structured routing decisions.

"""

In [195]:
from pydantic import BaseModel, Field
from typing import Optional, Dict, List


def has_cached_transcript(youtuber, video_id, window_size: int = 15, step_size: int = 3):
    """
    Check whether cached transcript chunks exist for a given YouTube video.

    The function looks for a JSON file named
        {youtuber}_{video_id}.json
    inside the directory:
        {CACHE_DATA_DIR}/{youtuber}_{window_size}_{step_size}/

    Args:
        youtuber (str): The YouTuber's name used for folder and file naming.
        video_id (str): The YouTube video ID.
        window_size (int): Sliding window size used during transcript chunking.
        step_size (int): Step size used during transcript chunking.

    Returns:
        bool: True if the cached transcript file exists, False otherwise.
    """

    cache_dir = CACHE_DATA_DIR / Path(f"{youtuber}_{window_size}_{step_size}_Chinese")
    file_path = cache_dir / f"{youtuber}_{video_id}.json"

    return file_path.exists()

import json

def list_cached_videos(youtuber: str, window_size: int = 15, step_size: int = 3) -> List[dict]:
    """
    List all cached video IDs for a given YouTuber.

    Args:
        youtuber (str): Name of the YouTuber.
        window_size (int): Sliding window size used during processing.
        step_size (int): Step size used during processing.

    Returns:
        List[dict]: List of title, summary, video IDs available in the cache for this YouTuber.
    """
    
    cache_dir = CACHE_DATA_DIR / f"{youtuber}_{window_size}_{step_size}_Chinese"

    if not cache_dir.exists():
        return []

    video_url = []
    for file in cache_dir.glob(f"{youtuber}_*.json"):
        with open(file, "r", encoding="utf-8") as f:
            chunks = json.load(f)
        summary = chunks[0]['summary']
        title = chunks[0]['title']
        # Extract video_id from filename
        name = file.stem  # e.g., "heyitsmindy_9QSmr_2EGXM"
        parts = name.split(f"{youtuber}_")
        if len(parts) == 2:
            video = {
                'title': title,
                'summary': summary,
                'url': f"https://www.youtube.com/watch?v={parts[1]}"
            }
            video_url.append(video)
    return video_url

In [196]:
from service.youtube_extraction.src.youtube_extraction.youtube_process import YoutuberTranscriptProcessor

In [197]:
# yts = YoutuberTranscriptSearcher()
ytp = YoutuberTranscriptProcessor()

clarify_agent = Agent(
    name='Clarify_Agent',
    instructions=clarify_instruction,
    tools=[ytp.chunk_transcript, has_cached_transcript, list_cached_videos],
    model='gpt-4o-mini',
    output_type=ClarifyDecision
)
callback = NamedCallback(clarify_agent)

In [198]:
question = 'What is the best foundation that heyitsmindy recommend'
# question = 'help me understand the video for https://www.youtube.com/watch?v=2SvN45DKWFg from heyitsmindy'
# question = 'Help me understand http://www.youtube.com/watch?v=2SvN45DKWFg from heyitsmindy'
question = 'Help me understand  https://www.youtube.com/watch?v=KNR7lyrTThg from heyitsmindy'

# question = 'Do you have this video https://www.youtube.com/watch?v=KNR7lyrTThg from heyitsmindy?'

In [ ]:
callback = NamedCallback(clarify_agent)
result = await clarify_agent.run(
        user_prompt=question,
        event_stream_handler=callback,
    )

TOOL CALL (Clarify_Agent): has_cached_transcript({"youtuber":"heyitsmindy","video_id":"KNR7lyrTThg"})
TOOL CALL (Clarify_Agent): list_cached_videos({"youtuber":"heyitsmindy"})


In [202]:
result.output.print_agent_output()

=== Clarify Agent Output ===
Intent       : video_question
YouTuber     : heyitsmindy
Video ID     : KNR7lyrTThg
Topic        : None
In Cache  : True

Hey I can start answer your question!
Here are Available Videos (metadata):

1. Title  : Sephora Summary: What to Buy and What Not to!
   Summary: In this video, Mindy shares various products she has purchased from Sephora over the past few years, including lip balms, primers, foundations, concealers, setting powders, blushes, and lip colors. She emphasizes the use and effects of several products, especially recommending the Topicals lip balm, Tower 28's SOS primer spray, and Natasha Denona's Hy-Glam foundation, believing these products are suitable for Asian skin tones and worth purchasing during sales. The video also mentions products she does not recommend, such as Charlotte Tilbury's Airbrush foundation and certain brands of blushes, highlighting their shortcomings in coverage and finish. At the end of the video, Mindy hopes viewers 

In [ ]:
result.output.user_intent

'video_question'

In [176]:
result.output.in_cache

False

In [177]:
result.all_messages()

[ModelRequest(parts=[UserPromptPart(content='Help me understand  https://www.youtube.com/watch?v=KNR7lyrTThg from heyitsmindy', timestamp=datetime.datetime(2025, 11, 27, 18, 39, 3, 380933, tzinfo=datetime.timezone.utc))], instructions='You are the Clarify Agent for a Beauty & YouTube Review Search System.\n\nYour ONLY responsibilities are:\n1. Understanding the user\'s message.\n2. Determining whether to use cached transcript data or fetch/index new data.\n3. Routing the flow correctly using the ClarifyDecision schema.\n\nYou MUST NOT:\n- Answer beauty questions.\n- Recommend products.\n- Summarize any video.\n- Pull transcript content.\nYou ONLY perform routing and decision-making.\n\n====================================================\nPRIMARY RULE — Cache First\n====================================================\nAlways check if the transcript for the video is in the cache **before doing anything else**.\n- If the transcript exists in cache → do NOT call chunk_transcript.\n- If t

In [182]:
def print_agent_output(decision: ClarifyDecision):
    """
    Print the full Clarify Agent output in a readable format.

    Args:
        decision (ClarifyDecision): The ClarifyDecision object returned by the agent.
    """
    print("=== Clarify Agent Output ===")
    print(f"Intent       : {decision.user_intent}")
    print(f"YouTuber     : {decision.youtuber}")
    print(f"Video ID     : {decision.video_id}")
    print(f"Topic        : {decision.topic}")
    # print(f"Add Data     : {decision.add_to_cache}")
    print(f"In Cache  : {decision.in_cache}\n")

    # if decision.ask_for_video:
    
    if True:
        
        print("Hey I can start answer your question!\nHere are Available Videos (metadata):\n")
        for idx, video in enumerate(decision.available_videos, start=1):
            print(f"{idx}. Title  : {video.title}")
            print(f"   Summary: {video.summary}")
            print(f"   URL    : {video.url}\n")
    elif decision.available_videos:
        print("Available Video IDs:")
        for vid in decision.available_videos:
            print(f"- {vid}")
    else:
        print("No available videos to display.")
    print("============================\n")
    
print_agent_output(result.output)

=== Clarify Agent Output ===
Intent       : video_question
YouTuber     : heyitsmindy
Video ID     : KNR7lyrTThg
Topic        : None
In Cache  : False

Hey I can start answer your question!
Here are Available Videos (metadata):

1. Title  : Sephora Summary: What to Buy and What Not to!
   Summary: In this video, Mindy shares various products she has purchased from Sephora over the past few years, including lip balms, primers, foundations, concealers, setting powders, blushes, and lip colors. She emphasizes the use and effects of several products, especially recommending the Topicals lip balm, Tower 28's SOS primer spray, and Natasha Denona's Hy-Glam foundation, believing these products are suitable for Asian skin tones and worth purchasing during sales. The video also mentions products she does not recommend, such as Charlotte Tilbury's Airbrush foundation and certain brands of blushes, highlighting their shortcomings in coverage and finish. At the end of the video, Mindy hopes viewers

In [214]:
from service.agent.src.agent.clarify_agent import create_clarify_agent

In [215]:
clarify_agent = create_clarify_agent()

In [216]:
callback = NamedCallback(clarify_agent)
question = 'Help me understand  https://www.youtube.com/watch?v=KNR7lyrTThg from heyitsmindy'

result = await clarify_agent.run(
        user_prompt=question,
        event_stream_handler=callback,
    )

TOOL CALL (Clarify_Agent): has_cached_transcript({"youtuber":"heyitsmindy","video_id":"KNR7lyrTThg"})
TOOL CALL (Clarify_Agent): chunk_transcript({"video_id":"KNR7lyrTThg"})
Processing... KNR7lyrTThg
TOOL CALL (Clarify_Agent): list_cached_videos({"youtuber":"heyitsmindy"})


In [218]:
result.output.print_agent_output()

=== Clarify Agent Output ===
Intent       : video_question
YouTuber     : heyitsmindy
Video ID     : KNR7lyrTThg
Topic        : None
In Cache  : False

Hey I can start answer your question!
Here are Available Videos (metadata):

1. Title  : Sephora Summary: What to Buy and What Not to!
   Summary: In this video, Mindy shares various products she has purchased from Sephora over the past few years, including lip balms, primers, foundations, concealers, setting powders, blushes, and lip colors. She emphasizes the use and effects of several products, especially recommending the Topicals lip balm, Tower 28's SOS primer spray, and Natasha Denona's Hy-Glam foundation, believing these products are suitable for Asian skin tones and worth purchasing during sales. The video also mentions products she does not recommend, such as Charlotte Tilbury's Airbrush foundation and certain brands of blushes, highlighting their shortcomings in coverage and finish. At the end of the video, Mindy hopes viewers

In [183]:
# orchestrator